<a href="https://colab.research.google.com/github/A00785001/TC5035/blob/main/01_Pre_processing_MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROS Bag Camera Data Extractor & MobileNet V2 Preprocessor
Extract camera images from ROS bags and prepare them for MobileNet V2 feature extraction

## Section 1: Extract Images from ROS Bag

In [ ]:
# Install required packages
!pip install bagpy

In [ ]:
# Import libraries
from bagpy import bagreader
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from glob import glob
import json
import csv
from datetime import datetime

print("Libraries loaded successfully!")

In [ ]:
# Upload your ROS bag file (for Colab)
from google.colab import files
uploaded = files.upload()
bag_file = list(uploaded.keys())[0]
print(f"Uploaded: {bag_file}")

In [ ]:
# Read the bag file
bag = bagreader(bag_file)
print(f"Opened bag file: {bag_file}")

In [ ]:
# List all topics in the bag
print("Topics in bag:")
print(bag.topic_table)

In [ ]:
# Specify your camera topic
camera_topic = "/camera/image_raw"
print(f"Using camera topic: {camera_topic}")

In [ ]:
# Extract images from the topic
print(f"Extracting images from topic: {camera_topic}")
image_data = bag.message_by_topic(camera_topic)
print(f"Images extracted to: {image_data}")

In [ ]:
# Get the folder where images were extracted
bag_name = os.path.splitext(bag_file)[0]
image_folder = os.path.join(bag_name, camera_topic.replace('/', '-')[1:])

# List all image files
image_files = sorted(glob(os.path.join(image_folder, '*.jpg')) +
                     glob(os.path.join(image_folder, '*.png')))

print(f"Found {len(image_files)} images")
print(f"Image folder: {image_folder}")

In [ ]:
# Display sample images
sample_images = []
for img_path in image_files[:6]:
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sample_images.append(img_rgb)

# Plot sample images
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()
for i, img in enumerate(sample_images):
    axes[i].imshow(img)
    axes[i].set_title(f"Image {i+1}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

if sample_images:
    print(f"Original image shape: {sample_images[0].shape}")

## Section 2: Process Images for MobileNet V2 Feature Extraction

**MobileNet V2 Requirements:**
- Input size: 224x224x3
- Pixel scaling to [-1, 1] will be done during feature extraction
- Here we only resize and save images

In [ ]:
# MobileNet V2 input parameters
IMG_SIZE = 224
TARGET_SIZE = (IMG_SIZE, IMG_SIZE)
JPEG_QUALITY = 95

print(f"Target image size: {TARGET_SIZE}")
print(f"JPEG quality: {JPEG_QUALITY}")

In [ ]:
# Create output folder structure
OUTPUT_DIR = "processed_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Created output directory: {OUTPUT_DIR}/")
print("Images will be saved with metadata (CSV + JSON)")

In [ ]:
# Extract timestamps from bagpy (if available)
# Try to read timestamp data from CSV file that bagpy creates
csv_file = image_data if isinstance(image_data, str) and image_data.endswith('.csv') else None
timestamps = []

if csv_file and os.path.exists(csv_file):
    try:
        import pandas as pd
        df = pd.read_csv(csv_file)
        if 'Time' in df.columns:
            timestamps = df['Time'].tolist()
            print(f"Loaded {len(timestamps)} timestamps from bag")
    except:
        print("Could not load timestamps from CSV")

# If no timestamps available, use sequential numbering
if len(timestamps) != len(image_files):
    print("Using sequential frame IDs instead of timestamps")
    timestamps = list(range(len(image_files)))

In [ ]:
# Function to preprocess and save images
def preprocess_and_save_image(img_path, save_path, target_size=(224, 224), quality=95):
    """
    Preprocess image for MobileNet V2:
    1. Load image
    2. Resize to 224x224
    3. Save as JPEG

    Returns: (success, original_width, original_height, file_size_kb)
    """
    try:
        # Load image
        img = Image.open(img_path)
        original_size = img.size  # (width, height)

        # Resize to target size
        img_resized = img.resize(target_size, Image.LANCZOS)

        # Save preprocessed image
        img_resized.save(save_path, 'JPEG', quality=quality)

        # Get file size
        file_size_kb = os.path.getsize(save_path) / 1024

        return True, original_size[0], original_size[1], file_size_kb
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return False, 0, 0, 0

print("Preprocessing function ready!")

In [ ]:
# Process all images and collect metadata
print("Processing images...")

metadata_list = []
total_size_kb = 0
failed_count = 0

for i, img_path in enumerate(image_files):
    # Generate output filename
    output_filename = f"img_{i:05d}.jpg"
    output_path = os.path.join(OUTPUT_DIR, output_filename)

    # Process and save image
    success, orig_w, orig_h, file_size = preprocess_and_save_image(
        img_path, output_path, TARGET_SIZE, JPEG_QUALITY
    )

    if success:
        # Get timestamp info
        timestamp = timestamps[i] if i < len(timestamps) else i

        # Parse timestamp if it's a float (ROS timestamp)
        if isinstance(timestamp, float):
            timestamp_sec = int(timestamp)
            timestamp_nsec = int((timestamp - timestamp_sec) * 1e9)
        else:
            timestamp_sec = timestamp
            timestamp_nsec = 0

        # Store metadata
        metadata_list.append({
            'filename': output_filename,
            'timestamp': timestamp,
            'timestamp_sec': timestamp_sec,
            'timestamp_nsec': timestamp_nsec,
            'frame_id': i,
            'original_width': orig_w,
            'original_height': orig_h,
            'file_size_kb': round(file_size, 2)
        })

        total_size_kb += file_size
    else:
        failed_count += 1

    # Progress update
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{len(image_files)} images")

print(f"\n✓ Completed: {len(metadata_list)} images processed")
print(f"✗ Failed: {failed_count} images")
print(f"Total dataset size: {total_size_kb/1024:.2f} MB")

In [ ]:
# Save CSV metadata (per-image data)
csv_path = os.path.join(OUTPUT_DIR, 'metadata.csv')

with open(csv_path, 'w', newline='') as csvfile:
    fieldnames = ['filename', 'timestamp', 'timestamp_sec', 'timestamp_nsec',
                  'frame_id', 'original_width', 'original_height', 'file_size_kb']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in metadata_list:
        writer.writerow(row)

print(f"✓ CSV metadata saved: {csv_path}")

In [ ]:
# Calculate statistics
if metadata_list:
    avg_file_size = sum(m['file_size_kb'] for m in metadata_list) / len(metadata_list)

    # Calculate FPS if timestamps are available
    if len(metadata_list) > 1 and isinstance(metadata_list[0]['timestamp'], float):
        time_diff = metadata_list[-1]['timestamp'] - metadata_list[0]['timestamp']
        fps_actual = len(metadata_list) / time_diff if time_diff > 0 else 0
    else:
        fps_actual = 0
else:
    avg_file_size = 0
    fps_actual = 0

In [ ]:
# Create JSON metadata (dataset-level info)
json_metadata = {
    "dataset_metadata": {
        "creation_date": datetime.now().isoformat(),
        "ros_bag_info": {
            "source_file": bag_file,
            "bag_duration_sec": metadata_list[-1]['timestamp'] - metadata_list[0]['timestamp'] if len(metadata_list) > 1 and isinstance(metadata_list[0]['timestamp'], float) else 0,
            "bag_start_time": metadata_list[0]['timestamp'] if metadata_list else 0,
            "bag_end_time": metadata_list[-1]['timestamp'] if metadata_list else 0
        },
        "camera_info": {
            "topic": camera_topic,
            "encoding": "bgr8",
            "frame_rate_hz": round(fps_actual, 2),
            "camera_model": "unknown"
        },
        "processing_info": {
            "target_size": [IMG_SIZE, IMG_SIZE],
            "resize_method": "LANCZOS",
            "jpeg_quality": JPEG_QUALITY,
            "total_images_processed": len(metadata_list),
            "processing_script": "rosbag_camera_extractor_v1.ipynb"
        }
    },
    "optional_camera_calibration": {
        "camera_matrix": None,
        "distortion_coefficients": None,
        "calibration_available": False
    },
    "sensor_fusion_notes": {
        "time_synchronization": "ROS timestamps preserved in CSV",
        "coordinate_frame": "camera_optical_frame",
        "notes": "Images ready for MobileNet V2 feature extraction. Apply preprocess_input() before inference."
    },
    "statistics": {
        "fps_actual": round(fps_actual, 2),
        "dropped_frames": failed_count,
        "avg_file_size_kb": round(avg_file_size, 2),
        "total_dataset_size_mb": round(total_size_kb / 1024, 2)
    }
}

# Save JSON metadata
json_path = os.path.join(OUTPUT_DIR, 'dataset_info.json')
with open(json_path, 'w') as jsonfile:
    json.dump(json_metadata, jsonfile, indent=2)

print(f"✓ JSON metadata saved: {json_path}")

In [ ]:
# Display summary
print("\n" + "="*50)
print("DATASET PROCESSING COMPLETE")
print("="*50)
print(f"Output directory: {OUTPUT_DIR}/")
print(f"Total images: {len(metadata_list)}")
print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Dataset size: {total_size_kb/1024:.2f} MB")
print(f"Average FPS: {fps_actual:.2f}" if fps_actual > 0 else "FPS: N/A")
print(f"\nMetadata files:")
print(f"  - {csv_path}")
print(f"  - {json_path}")
print("\n" + "="*50)

In [ ]:
# Display sample processed images
sample_processed = []
output_images = sorted(glob(os.path.join(OUTPUT_DIR, '*.jpg')))[:6]

for img_path in output_images:
    img = Image.open(img_path)
    sample_processed.append(np.array(img))

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()
for i, img in enumerate(sample_processed):
    axes[i].imshow(img)
    axes[i].set_title(f"Processed - {img.shape}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## Next Steps: Feature Extraction with MobileNet V2

Your images are now ready for MobileNet V2 feature extraction. When you're ready to extract features:

```python
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd

# Load MobileNet V2 (without top classification layer)
model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

# Load metadata
df = pd.read_csv('processed_images/metadata.csv')

# Extract features for each image
features_list = []
for idx, row in df.iterrows():
    img_path = f"processed_images/{row['filename']}"
    
    # Load and preprocess
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_preprocessed = preprocess_input(img_array)  # Scale to [-1, 1]
    
    # Extract features
    features = model.predict(np.expand_dims(img_preprocessed, axis=0))
    features_list.append(features[0])

# features_list now contains feature vectors for sensor fusion
```

**Note:** The `preprocess_input()` function applies the [-1, 1] scaling required by MobileNet V2.